In [14]:
import sys
from os import path as osp
from datetime import datetime
from tqdm import tqdm
from skimage import io, transform

# torch imports
import torch
from torchvision import transforms
from torch.utils.data.dataloader import DataLoader

# root path of project
from os import path as osp
import sys

# get root directory
import re
reg = '^.*/AquaPose'
project_root = re.findall(reg, osp.dirname(osp.abspath(sys.argv[0])))[0]
sys.path.append(project_root)

from lib.dataset.PoseDataset import PoseDataset
from lib.dataset.custom_transforms import DictToTensor
from lib.models.keypoint_rcnn import get_resnet50_pretrained_model


In [15]:
from glob import glob

model = get_resnet50_pretrained_model()
weight_dir = osp.join(project_root, 'weights')

weight_files = glob(osp.join(weight_dir,'*'))
print(weight_files)

['/mnt/c/Users/Vande/OneDrive - ugentbe/Ingenieurswetenschappen/_master/thesis/AquaPose/weights/weights_4_box_and_key_predictor.wth', '/mnt/c/Users/Vande/OneDrive - ugentbe/Ingenieurswetenschappen/_master/thesis/AquaPose/weights/weights_4_epoch.wth', '/mnt/c/Users/Vande/OneDrive - ugentbe/Ingenieurswetenschappen/_master/thesis/AquaPose/weights/weights_4_finally_fixed.wth', '/mnt/c/Users/Vande/OneDrive - ugentbe/Ingenieurswetenschappen/_master/thesis/AquaPose/weights/weights_4_finally_fixed_interrupted.wth', '/mnt/c/Users/Vande/OneDrive - ugentbe/Ingenieurswetenschappen/_master/thesis/AquaPose/weights/weights_4_only_box.wth', '/mnt/c/Users/Vande/OneDrive - ugentbe/Ingenieurswetenschappen/_master/thesis/AquaPose/weights/weights_breaststroke_epoch0', '/mnt/c/Users/Vande/OneDrive - ugentbe/Ingenieurswetenschappen/_master/thesis/AquaPose/weights/weights_breaststroke_epoch0.wth', '/mnt/c/Users/Vande/OneDrive - ugentbe/Ingenieurswetenschappen/_master/thesis/AquaPose/weights/weights_only_box.w

In [16]:
model.load_state_dict(torch.load(weight_files[-1]))

#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')



<All keys matched successfully>

In [17]:
annotation_keypoints = {
            '0': "Nose",
            '1': "left_eye",
            '2': "right_eye",
            '3': "left_ear",
            '4': "right_ear",
            '5': "Shoulder_Close",
            '6': "Shoulder_Far",
            '7': "Elbow_Close",
            '8': "Elbow_Far",
            '9': "Wrist_Close",
            '10': "Wrist_Far",
            '11': "Hip_Close",
            '12': "Hip_Far",
            '13': "Knee_Close",
            '14': "Knee_Far",
            '15': "Ankle_Close",
            '16': "Ankle_Far",
}

In [18]:
demo_dir = osp.join(project_root, 'demo_data')
image_files = glob(osp.join(demo_dir, '*'))



In [19]:
image = io.imread(image_files[0])
image_tensor = transforms.ToTensor()(image)
model.eval()
prediction = model([image_tensor])